In [1]:
import pandas as pd
import numpy as np
import dask 
import dask.array as da
import cv2
import os
import glob
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
from tqdm import tqdm

In [3]:
from keras.callbacks import EarlyStopping
from keras import layers
from keras import Input
from keras.models import Sequential
import keras
from keras import optimizers
from keras.preprocessing.image import array_to_img, img_to_array, load_img, ImageDataGenerator

Using TensorFlow backend.


In [4]:
from keras.preprocessing.image import load_img, array_to_img, img_to_array

# data

In [5]:
train_dir = './UTKface_inthewild/train/train_sex/_'
validation_dir = 'UTKface_inthewild/validation/validation_sex/_'

In [6]:
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

In [47]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(200, 200),
    batch_size=22,
    class_mode='binary'
)

Found 22506 images belonging to 2 classes.


In [20]:
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(200, 200),
    batch_size=20,
    class_mode='binary'
)

Found 3289 images belonging to 2 classes.


# model

## conv base

In [35]:
from keras.applications import Xception

In [36]:
conv_base = Xception(
    weights = 'imagenet',
    include_top = False,
    input_shape=(200, 200, 3)
)

In [26]:
conv_base.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 200, 200, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 99, 99, 32)   864         input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 99, 99, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 99, 99, 32)   0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [37]:
for layer in conv_base.layers[:108]:
    layer.trainable=False

## create base

In [43]:
model = Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.BatchNormalization())
# model.add(layers.Dropout(0.5))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.BatchNormalization())
# model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

In [30]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, 7, 7, 2048)        20861480  
_________________________________________________________________
flatten_2 (Flatten)          (None, 100352)            0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 100352)            401408    
_________________________________________________________________
dense_3 (Dense)              (None, 256)               25690368  
_________________________________________________________________
batch_normalization_8 (Batch (None, 256)               1024      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 257       
Total params: 46,954,537
Trainable params: 33,757,665
Non-trainable params: 13,196,872
_______________________________________________________

In [44]:
model.compile(
    loss='binary_crossentropy',
    optimizer=optimizers.Adam(lr=1e-5),
    metrics=['acc']
)

In [45]:
callbacks_list = [
#     keras.callbacks.EarlyStopping(
#         monitor="val_acc",         
#         patience=10               
#     ),
    keras.callbacks.ModelCheckpoint(
        filepath="my_model.h5",     
        monitor='val_acc',        
        save_best_only=True,       
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_acc',       
        factor=0.5,                 
        patience=5                 
    )
]

# learn

In [48]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=1000,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=16
)

Epoch 1/30
1000/1000 [==============================] - 4013s 4s/step - loss: 0.3586 - acc: 0.8375 - val_loss: 0.5755 - val_acc: 0.8219
Epoch 2/30
 397/1000 [==========>...................] - ETA: 2:00:15 - loss: 0.2325 - acc: 0.8998

KeyboardInterrupt: 